In [6]:
from prelu_cnn import CNN

# Load the test dataset and process a single example
from datasets import load_dataset
from prelu_cnn import preprocess_images

# Load ImageNet dataset
dataset = load_dataset("imagenet-1k")

ds = dataset['train'].select(range(2))
processed_examples = ds.map(preprocess_images, batched=True)
processed_examples


imagenet-1k.py:   0%|          | 0.00/4.58k [00:00<?, ?B/s]

classes.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

/Users/chrisobrien/Documents/Development/model-examples/.venv/lib/python3.13/site-packages/huggingface_hub/file_download.py:799: UserWarning: Not enough free disk space to download the file. The expected file size is: 29138.26 MB. The target location /Users/chrisobrien/.cache/huggingface/hub/datasets--imagenet-1k/blobs only has 25261.70 MB free disk space.
  warnings.warn(


train_images_0.tar.gz:   0%|          | 0.00/29.1G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [36]:
import torch
inputs = torch.tensor(processed_examples['pixel_values'])
labels = torch.tensor(processed_examples['labels'])
inputs.shape, labels.shape


(torch.Size([2, 3, 32, 32]), torch.Size([2]))

In [37]:
model = CNN(use_prelu=False, use_builtin_conv=False)
outputs = model(inputs)
outputs

tensor([[-0.1768, -0.0178,  0.3206,  0.1032, -0.2640, -0.0082, -0.0429,  0.1991,
         -0.2714,  0.3188],
        [-0.0204,  0.0128,  0.1151,  0.2245,  0.1191,  0.0656, -0.1534,  0.2452,
         -0.1060, -0.0903]], grad_fn=<AddmmBackward0>)

In [45]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions =np.argmax(outputs.detach().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.numpy())
print("Correct predictions:", (predictions == labels.numpy()).sum(), "out of", len(predictions))


Predictions: [2 7]
Labels: [0 6]
Correct predictions: 0 out of 2


In [40]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(2.5113, grad_fn=<NllLossBackward0>)